In [12]:
#网页模式抓取二手房成交记录
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os

OriginURL = r"https://sh.lianjia.com/chengjiao/"
Header ={
"Host": "sh.lianjia.com",
"Connection": "keep-alive",
"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36",
"Accept": "image/webp,image/apng,image/*,*/*;q=0.8",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "zh-CN,zh;q=0.9"}


def get_block_list(url):
    _connection = requests.get(url)
    _BS = BeautifulSoup(_connection.text, "lxml")
    _BlockListOrigin = _BS.find("div", class_="position").find_all("a", href=re.compile("/chengjiao/\w+/"), rel=False)
    _BlockList = ["https://sh.lianjia.com" + i["href"] for i in _BlockListOrigin]
    return _BlockList


def get_block_list_of_a_city(url):
    _BlockList = []
    _connection = requests.get(url)
    _BS = BeautifulSoup(_connection.text, "lxml")
    _DistrictListOrigin = _BS.find_all("a", href=re.compile("/chengjiao/\w+/"), title=re.compile("上海"))
    _DistrictList = ["https://sh.lianjia.com" + i["href"] for i in _DistrictListOrigin]
    print("Fetched the {}".format(url))
    for i in _DistrictList:
        templist = get_block_list(i)
        _BlockList = _BlockList + templist
        print("Fetched the {}".format(i))
    _BlockList = set(_BlockList)-set(_DistrictList)
    return _BlockList


def save_file(data, filename, path=os.getcwd(), filetype="txt"):
    """

    :param data:
    :param filename:
    :param path:
    :param filetype:
    :return:
    """
    if filetype == 'txt':
        with open("{0}/{1}".format(path, filename), "w") as f:
            for i in data:
                f.write("{}\n".format(i))
                
                
save_file(get_block_list_of_a_city(OriginURL),"BlockList.txt")

In [84]:
#API方式地图抓取
import json
import time
import hashlib
import requests

group_type = {
    "district":0.3,     #行政区
    "bizcircle":0.08,    #商圈
    "community":0.01     #居民小区
}
31.24916171001514  
b= {"city_id": "310000",
     "group_type": "bizcircle",
     "max_lat": round(31.21516171001514, 6),
     "min_lat": round(31.21516171001514, 6) - group_type[b["group_type"]],
     "max_lng": round(121.49389948569473, 6),
     "min_lng": round(121.49389948569473, 6) - group_type[b["group_type"]],
   }

def get_md5(txt):
    """md5加密函数"""
    m = hashlib.md5()
    m.update(txt.encode('utf-8'))
    return m.hexdigest()

def get_line_site(url):
    """请求链接"""
    headers = {'Accept': '*/*',
               'Accept-Encoding': 'gzip, deflate, br',
               'Accept-Language': 'zh-CN,zh;q=0.9',
               'Connection': 'keep-alive',
               'Host': 'ajax.lianjia.com',
               'Referer': 'https://gz.lianjia.com/ditu/',
               'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 '
                             '(KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'}
    r = requests.get(url, headers=headers)
    return r

def get_url(bbb):
    """拼接链接"""
    url ="https://ajax.lianjia.com/map/search/ershoufang/?city_id={city_id}&group_type={group_type}&" \
         "max_lat={max_lat}&min_lat={min_lat}&max_lng={max_lng}&min_lng={min_lng}&" \
         "filters=%7B%7D&request_ts={request_ts}&source=ljpc&" \
         "authorization={authorization}"
    request_ts = int(time.time() * 1000)
    md5_data = "vfkpbin1ix2rb88gfjebs0f60cbvhedlcity_id={city_id}group_type={group_type}max_lat={max_lat}" \
             "max_lng={max_lng}min_lat={min_lat}min_lng={min_lng}request_ts={request_ts}".format(
        city_id=bbb["city_id"],
        group_type=bbb["group_type"],
        max_lat=bbb["max_lat"],
        max_lng=bbb["max_lng"],
        min_lat=bbb["min_lat"],
        min_lng=bbb["min_lng"],
        request_ts=request_ts)
    authorization = get_md5(md5_data)
    url = url.format(request_ts=request_ts, authorization=authorization,city_id=bbb["city_id"],
        group_type=bbb["group_type"],
        max_lat=bbb["max_lat"],
        max_lng=bbb["max_lng"],
        min_lat=bbb["min_lat"],
        min_lng=bbb["min_lng"])
    return url

def getres(bb):
    line_url = get_url(bb)
    res = get_line_site(line_url)
    items = res.json()['data']
    return items 

getres(b)


{'list': {'613000343': {'id': 613000343,
   'name': '康健',
   'longitude': 121.423904,
   'latitude': 31.164627,
   'border': '121.440684,31.173781;121.44044,31.171378;121.440226,31.170019;121.439957,31.169092;121.439454,31.168026;121.438922,31.167184;121.438448,31.166535;121.437988,31.166025;121.435088,31.16319;121.43145,31.159737;121.427585,31.155982;121.424109,31.152598;121.421189,31.149771;121.418459,31.147028;121.415728,31.152861;121.413653,31.157721;121.41156,31.162665;121.409235,31.168033;121.410473,31.168551;121.411633,31.168891;121.413248,31.169216;121.418344,31.170281;121.420571,31.170776;121.421362,31.171092;121.422125,31.171455;121.423958,31.172506;121.424668,31.172838;121.425431,31.173016;121.428304,31.173425;121.433166,31.174089;121.435634,31.17443;121.438114,31.174515;121.440799,31.174638',
   'unit_price': 71120,
   'count': 579},
  '611900099': {'id': 611900099,
   'name': '建国西路',
   'longitude': 121.463357,
   'latitude': 31.213498,
   'border': '121.456539,31.216949;1

In [85]:
#获取城市纬度
import json

ak = "dASz7ubuSpHidP1oQWKuAK3q"
def getCor(city):
    url = "http://api.map.baidu.com/geocoder/v2/?address={city}&output=json&ak={ak}".format(city=city,ak=ak)
    res = requests.get(url).json()["result"]["location"]
    return res

def getCode(city):
    Cor = getCor(city)
    url2 = "http://api.map.baidu.com/geocoder/v2/?location={lat},{lng}&output=json&pois=1&ak={ak}".format(lat=Cor["lat"], lng=Cor["lng"],ak=ak)
    print(url2)
    res = requests.get(url2).json()
    return res

getCor("北京")
    

{'lng': 116.39564503787867, 'lat': 39.92998577808024}

In [46]:
#生成城市列表
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

res = requests.get("https://sh.lianjia.com/city/")
work = BeautifulSoup(res.text, "lxml")
res = set([(i["href"], i.string) for i in work.find_all("a",href=re.compile(r"(?<=https://)\w+(?=\.lianjia.com/)"),target=False)][:-1])
citycodemapping = pd.read_csv(r"C:\Users\wenpeiyu\Desktop\citycodemapping.csv")
dfcitylist = pd.DataFrame({
    "city": [i[1] for i in res],
    "cityurl": [i[0] for i in res]})
dfCityList = pd.merge(dfcitylist,citycodemapping,how="inner",on=["city"])
lng = [getCor(i+"市")['lng'] for i in dfCityList.city]
lat = [getCor(i+"市")['lat'] for i in dfCityList.city]
city = [i for i in dfCityList.city]
dfCityCor = pd.DataFrame({
    "city": city,
    "lat": lat,
    "lng":lng
})
dfCityList = pd.merge(dfCityList,dfCityCor,how="inner",on=["city"])